In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Load datasets
df_songs = pd.read_csv('../data/features_30_sec.csv')
df_segments = pd.read_csv('../data/features_3_sec.csv')

# Prepare training and testing data
X = df_songs
y = df_songs['label']

X_train_songs, X_test_songs, y_train_songs, y_test_songs = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_filenames = X_train_songs['filename']
test_filenames = X_test_songs['filename']

train_mask = (df_segments['filename'].str.extract(r'(.*\.\d+)\.\d+\.wav')[0] + '.wav').isin(train_filenames)
test_mask = (df_segments['filename'].str.extract(r'(.*\.\d+)\.\d+\.wav')[0] + '.wav').isin(test_filenames)

df_train_segments = df_segments[train_mask].reset_index(drop=True)
df_test_segments = df_segments[test_mask].reset_index(drop=True)

X_train_segments = df_train_segments.drop(columns=['filename', 'length', 'label'])
y_train_segments = df_train_segments['label']

In [14]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=10000))
])

In [13]:
# Define parameter grid for GridSearchCV
param_grid = {
    'logreg__C': [0.1, 1, 10, 100],
    'logreg__solver': ['liblinear', 'lbfgs', 'newton-cg']
}

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train_segments, y_train_segments)

# Print best parameters and best score
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")

KeyboardInterrupt: 

In [ ]:
# Use the best estimator to predict on test set
best_clf = grid_search.best_estimator_
X_test_segments = df_test_segments.drop(columns=['filename', 'length', 'label'])
y_test_segments = df_test_segments['label']

y_pred = best_clf.predict(X_test_segments)

# Define majority vote function
def majority_vote(predictions):
    return predictions.mode().iloc[0]

df_test_segments['predicted_label'] = y_pred

df_test_segments['song'] = df_test_segments['filename'].str.extract(r'(.*\.\d{5})')[0] + '.wav'
song_predictions = df_test_segments.groupby('song')['predicted_label'].apply(majority_vote)

song_true_labels = X_test_songs.set_index('filename').loc[song_predictions.index, 'label']
song_pred_labels = song_predictions

# Print classification report and accuracy score
print("Classification Report:")
print(classification_report(song_true_labels, song_pred_labels))

print("Accuracy Score:")
print(accuracy_score(song_true_labels, song_pred_labels))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- predicted_label
- song
